# BiteMe | Preprocessing

 - Preprocessing pipeline
 - Train/test split
 - Augmentations
 - Write augmented images into train/label/img and test/label/img
 - Write metadata including processed images. Write images first with augs, then rename images to hash, then create metadata.
 
 